<a href="https://colab.research.google.com/github/yequalsmxplusc/optimisation-sem6/blob/main/Asssignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dataset provided as follows:

### Comments explaining each column
### Column 1: Number of generator (n=13)
### Column 2: Minimum real power generation limit
### Column 3: Maximum real power generation limit (P_gmax)
### Column 4: Constant term a(i)
### Column 5: Linear term b(i)
### Column 6: Quadratic term d(i)

In [1]:
import numpy as np

# Define the data
pd = 2520

tdata = [
    [1, 0, 680, 550, 8.10, 0.00028],
    [2, 0, 360, 309, 8.10, 0.00056],
    [3, 0, 360, 307, 8.10, 0.00056],
    [4, 60, 180, 240, 7.74, 0.00324],
    [5, 60, 180, 240, 7.74, 0.00324],
    [6, 60, 180, 240, 7.74, 0.00324],
    [7, 60, 180, 240, 7.74, 0.00324],
    [8, 60, 180, 240, 7.74, 0.00324],
    [9, 60, 180, 240, 7.74, 0.00324],
    [10, 40, 120, 126, 8.60, 0.00284],
    [11, 40, 120, 126, 8.60, 0.00284],
    [12, 55, 120, 126, 8.60, 0.00284],
    [13, 55, 120, 126, 8.60, 0.00284]
]

# Convert the data to a NumPy array for easier manipulation
tdata = np.array(tdata)

# Print the data to verify
# print("pd:", pd)
# print("tdata:\n", tdata)

pd: 2520
tdata:
 [[1.00e+00 0.00e+00 6.80e+02 5.50e+02 8.10e+00 2.80e-04]
 [2.00e+00 0.00e+00 3.60e+02 3.09e+02 8.10e+00 5.60e-04]
 [3.00e+00 0.00e+00 3.60e+02 3.07e+02 8.10e+00 5.60e-04]
 [4.00e+00 6.00e+01 1.80e+02 2.40e+02 7.74e+00 3.24e-03]
 [5.00e+00 6.00e+01 1.80e+02 2.40e+02 7.74e+00 3.24e-03]
 [6.00e+00 6.00e+01 1.80e+02 2.40e+02 7.74e+00 3.24e-03]
 [7.00e+00 6.00e+01 1.80e+02 2.40e+02 7.74e+00 3.24e-03]
 [8.00e+00 6.00e+01 1.80e+02 2.40e+02 7.74e+00 3.24e-03]
 [9.00e+00 6.00e+01 1.80e+02 2.40e+02 7.74e+00 3.24e-03]
 [1.00e+01 4.00e+01 1.20e+02 1.26e+02 8.60e+00 2.84e-03]
 [1.10e+01 4.00e+01 1.20e+02 1.26e+02 8.60e+00 2.84e-03]
 [1.20e+01 5.50e+01 1.20e+02 1.26e+02 8.60e+00 2.84e-03]
 [1.30e+01 5.50e+01 1.20e+02 1.26e+02 8.60e+00 2.84e-03]]


Differential Evolution (DE) is a population-based optimization algorithm that is particularly effective for continuous optimization problems. Below is a Python implementation of the Differential Evolution algorithm applied to the given dataset for cost optimization. The goal is to minimize the cost function defined by the quadratic cost terms provided in the dataset.

Steps:

- Cost Function: The cost function calculates the total cost based on the quadratic cost terms provided in the dataset. It also ensures that the total power generated meets the demand (pd).
- Initialization: The initial population is generated randomly within the bounds specified by the minimum and maximum real power generation limits.
- Differential Evolution: The DE algorithm is implemented with mutation, crossover, and selection steps. The population is evolved over a specified number of generations.
- Additional Solutions: After evolving the initial population, an additional set of solutions is generated and combined with the evolved population to create a total of 40 solutions.

In [2]:
# Cost function
def cost_function(solution, tdata, pd):
    total_cost = 0
    total_power = 0
    for i in range(len(solution)):
        p = solution[i]
        a, b, c = tdata[i, 3], tdata[i, 4], tdata[i, 5]
        total_cost += a + b * p + c * p**2
        total_power += p
    if abs(total_power - pd) > 1e-5:  # Ensure the power demand is met
        return float('inf')
    return total_cost

# Differential Evolution parameters
pop_size = 20
generations = 2
F = 0.8  # Differential weight
CR = 0.9  # Crossover probability

# Initialize population
population = np.random.rand(pop_size, len(tdata))
for i in range(pop_size):
    for j in range(len(tdata)):
        population[i, j] = np.random.uniform(tdata[j, 1], tdata[j, 2])

# Differential Evolution algorithm
for gen in range(generations):
    for i in range(pop_size):
        # Select three random vectors different from the current vector
        idxs = [idx for idx in range(pop_size) if idx != i]
        a, b, c = population[np.random.choice(idxs, 3, replace=False)]

        # Mutation
        mutant_vector = np.clip(a + F * (b - c), tdata[:, 1], tdata[:, 2])

        # Crossover
        crossover = np.random.rand(len(tdata)) < CR
        if not np.any(crossover):
            crossover[np.random.randint(0, len(tdata))] = True
        trial_vector = np.where(crossover, mutant_vector, population[i])

        # Selection
        if cost_function(trial_vector, tdata, pd) < cost_function(population[i], tdata, pd):
            population[i] = trial_vector

    # Print the best cost in the current generation
    best_cost = min(cost_function(ind, tdata, pd) for ind in population)
    print(f"Generation {gen + 1}: Best Cost = {best_cost}")

# Generate 40 solutions by iterating over the first solution set
additional_population = np.random.rand(20, len(tdata))
for i in range(20):
    for j in range(len(tdata)):
        additional_population[i, j] = np.random.uniform(tdata[j, 1], tdata[j, 2])

# Combine the populations
combined_population = np.vstack((population, additional_population))

# Print the final population
print("Final Population:\n", combined_population)

Generation 1: Best Cost = inf
Generation 2: Best Cost = inf
Final Population:
 [[6.12547037e+01 2.02928136e+01 4.97125599e+01 1.16964054e+02
  1.73133839e+02 6.41412795e+01 1.76516154e+02 1.09953113e+02
  7.71969060e+01 6.34618867e+01 5.43120271e+01 1.13630590e+02
  1.04107254e+02]
 [3.34460524e+02 2.22319094e+02 5.06639896e+00 1.24267088e+02
  6.49752455e+01 1.38381796e+02 8.97754862e+01 1.34845492e+02
  1.01289478e+02 5.87146548e+01 1.06477704e+02 7.39614819e+01
  1.18943086e+02]
 [2.98027415e+02 3.03078504e+02 3.35749083e+02 6.17252041e+01
  7.92972779e+01 1.15305088e+02 1.25037003e+02 1.14006442e+02
  6.75419251e+01 4.54742383e+01 1.10118734e+02 7.84318008e+01
  9.76504830e+01]
 [5.50342755e+02 1.98934183e+02 1.07498512e+02 6.54954257e+01
  1.06710205e+02 1.07586559e+02 1.28486032e+02 1.30620012e+02
  1.09370843e+02 4.43488677e+01 6.42366073e+01 1.07159888e+02
  6.49707029e+01]
 [1.41900276e+02 2.15875175e+02 2.13153161e+02 9.92163207e+01
  9.26261161e+01 1.44200950e+02 1.17655216e

### Notes:
- The cost_function ensures that the total power generated meets the demand (pd). If the demand is not met, the cost is set to infinity.
- The DE parameters (F, CR, pop_size, generations) can be adjusted based on the problem requirements.
- The final population is printed to verify the results.